In [2]:
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.parse
from urllib.request import Request, urlopen
import time
import pandas as pd
import psycopg2

## 인스타그램 검색어로 찾기

In [3]:
search = input("검색어를 입력하세요 : " )
searching = str(search)
search = urllib.parse.quote(search)
url = 'https://www.instagram.com/explore/tags/'+str(search)+'/'
driver = webdriver.Chrome()

driver.get(url) #검색어 입력한 인스타그램 url 저장
time.sleep(6) #로딩 시간을 위한 속도조절 

SCROLL_PAUSE_TIME = 1.2  #인스타게시물 스크롤 속도 조절 ( 1.0 ~ 2.0까지 사양에 맞게 조절 )

## 로그인

In [5]:
driver.find_element(By.XPATH, '//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button/div').click()# 로그인 박스 클릭
username = input()
password = input()                                                                                  # 로그인 아이디,패스워드 입력 
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[1]/div/label/input').click()            # ID 공란 클릭
ID = driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[1]/div/label/input')
ID.send_keys(username)                                                                              # 아이디 입력
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[2]/div/label/input').click()            # password 공란 클릭
passkey=driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[2]/div/label/input')
passkey.send_keys(password)                                                                         # 패스워드 입력
time.sleep(1)                                                           
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div').click()                 # 로그인
time.sleep(5)                                                                                       # 로딩 대기 시간
driver.find_element(By.XPATH, '//*[@id="react-root"]/section/main/div/div/div/div/button').click()  # 로그인 이후 뜨는 정보저장 무시

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button/div"}
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00667413+2389011]
	Ordinal0 [0x005F9F61+1941345]
	Ordinal0 [0x004EC658+837208]
	Ordinal0 [0x005191DD+1020381]
	Ordinal0 [0x0051949B+1021083]
	Ordinal0 [0x00546032+1204274]
	Ordinal0 [0x00534194+1130900]
	Ordinal0 [0x00544302+1196802]
	Ordinal0 [0x00533F66+1130342]
	Ordinal0 [0x0050E546+976198]
	Ordinal0 [0x0050F456+980054]
	GetHandleVerifier [0x00819632+1727522]
	GetHandleVerifier [0x008CBA4D+2457661]
	GetHandleVerifier [0x006FEB81+569713]
	GetHandleVerifier [0x006FDD76+566118]
	Ordinal0 [0x00600B2B+1968939]
	Ordinal0 [0x00605988+1989000]
	Ordinal0 [0x00605A75+1989237]
	Ordinal0 [0x0060ECB1+2026673]
	BaseThreadInitThunk [0x7605FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77B77A7E+286]
	RtlGetAppContainerNamedObjectPath [0x7

## 게시글 클릭하여 본문과 댓글 크롤링

In [6]:
time.sleep(7)  
driver.find_element(By.CLASS_NAME,'_9AhH0').click()  # 첫게시글 클릭

In [7]:
post = []
post.append(driver.find_element(By.CLASS_NAME, '_7UhW9.xLCgt.MMzan.KV-D4.se6yk.T0kll').text)  # 인스타 본문 내용 저장하기
post

['💚\n📷Photo by @iamyonghoon\n🏕발할\n#helsport#헬스포츠\n#camping#캠핑\n#발할#김레#패스빅\n#바랑에르캠프#바랑에르돔\n#헬스포츠텐트\n#감성캠핑#밀리캠핑#텐풍\n#헬스포츠크루']

In [64]:
# 인스타 댓글 저장하기 (진행중)

for b in range(1,3):
    a = driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[%d]/div/li/div/div/div[2]/div[1]/span' % (b)).text
    print(a)

일단 레쓰고고이죠 🤭🤭🥰🥰
5월은 캠핑일정 중에 젤 바쁜날같아요ㅋㅋ


In [26]:
result=[]
for i in post:
    temp = i.replace('\n','')
    result.append(temp)
result

['💚📷Photo by @iamyonghoon🏕발할#helsport#헬스포츠#camping#캠핑#발할#김레#패스빅#바랑에르캠프#바랑에르돔#헬스포츠텐트#감성캠핑#밀리캠핑#텐풍#헬스포츠크루',
 '*広角で撮るとものすごい広いテントに見えますね😳GWは昨年に続き雨予報…キャンプ行ってもまたこもりっきりになるのかな😢天気予報が変わることを祈り続けます🙏#ratelworks #waltent2 #キャンピングカー #バンライフ #キャンプ #夫婦キャンプ #幕内レイアウト #車中泊 #キャンプギア #camp #camper #outdoors #vanlife #rv #캠핑 #露營 #Lager #kamp #acampar #キャンプ好きな人と繋がりたい',
 '#협찬”툴레의 관심은 늘한가지,사람들이 아끼는 것을 옮길 때 그것을 도와주는 무언가“좋은 가방이 되고 싶은게 아니라, 당신의 소중한 물건을잘 지켜주고 싶다. 라는‘Bring your life’ 툴레✨6년전 툴레의 가이드포스트 75,65부터 버선트60, 캡스톤40, 올트레일35,25, 지금의 토피오40에 이르기까지 무수한 추억을 안겨준 툴레입니다!⭐️이번 토피오 컬랙션의 40리터 배낭은 1박을 요하는3계절 경량캠핑에 최적화 되어있어요!그 중 차별화 되는 부분이 등판과 토르소 조절 부분🤗✔️메쉬등판과 몸의 윤곽을 따라 이어지는 벨트로 시원함을유지하고 움직임이 쾌적합니다!✔️토르소 조절이 무척 간단해 계절마다 복장에 따른미세 토르소 조절이 필요할 시에 완벽한 피팅감을 느낄 수 있는편안함을 제공합니다. (개인적으로 가장 좋았던 부분)✔️어깨부분의 폼은 40리터급에서는 특별히 신경을 쓴편안함을 느낄 수 있었습니다!개인적으로 툴레배낭은 돌바닥에 깔고 의자로 사용해도, 패킹된 상태로 어느 부분을 잡더라도, 그 내구성에 대해서는 이미 수년간 저희집에서는 검증이 된 배낭이죠🤗당신의 소중한 무언가를 지켜줄 또 하나의 친구 툴레 토피오!꼭 체크해 보세요!\U0001fae1Bringyourlife @thulekorea#thule #툴레 #툴레토피오 #토피오#캠핑 #백패킹 

#### 게시글의 다음 페이지 누르기 방식 2개 + 누르기 이후 게시글 수집

In [9]:
driver.find_element(By.XPATH,'/html/body/div[6]/div[2]/div/div/button').click() # 인스타 첫번째 게시글일 때 다음 페이지 클릭하기


In [19]:
driver.find_element(By.XPATH,'/html/body/div[6]/div[2]/div/div[2]/button').click() # 다음 페이지 넘어간 이후 다음으로 넘어가기

In [20]:
time.sleep(1)
post.append(driver.find_element(By.CLASS_NAME, '_7UhW9.xLCgt.MMzan.KV-D4.se6yk.T0kll').text)  # 인스타 본문 내용 저장하기
post

['💚\n📷Photo by @iamyonghoon\n🏕발할\n#helsport#헬스포츠\n#camping#캠핑\n#발할#김레#패스빅\n#바랑에르캠프#바랑에르돔\n#헬스포츠텐트\n#감성캠핑#밀리캠핑#텐풍\n#헬스포츠크루',
 '*\n広角で撮るとものすごい広いテントに見えますね😳\n\nGWは昨年に続き雨予報…キャンプ行ってもまたこもりっきりになるのかな😢\n\n天気予報が変わることを祈り続けます🙏\n\n#ratelworks #waltent2 #キャンピングカー #バンライフ #キャンプ #夫婦キャンプ #幕内レイアウト #車中泊 #キャンプギア #camp #camper #outdoors #vanlife #rv #캠핑 #露營 #Lager #kamp #acampar #キャンプ好きな人と繋がりたい',
 '#협찬\n”툴레의 관심은 늘한가지,\n사람들이 아끼는 것을 옮길 때 그것을 도와주는 무언가“\n\n좋은 가방이 되고 싶은게 아니라, 당신의 소중한 물건을\n잘 지켜주고 싶다. 라는\n\n‘Bring your life’ 툴레✨\n\n6년전 툴레의 가이드포스트 75,65부터 버선트60, 캡스톤40, 올트레일35,25, 지금의 토피오40에 이르기까지 무수한 추억을 안겨준 툴레입니다!\n\n⭐️이번 토피오 컬랙션의 40리터 배낭은 1박을 요하는\n3계절 경량캠핑에 최적화 되어있어요!\n\n그 중 차별화 되는 부분이 등판과 토르소 조절 부분🤗\n\n✔️메쉬등판과 몸의 윤곽을 따라 이어지는 벨트로 시원함을\n유지하고 움직임이 쾌적합니다!\n\n✔️토르소 조절이 무척 간단해 계절마다 복장에 따른\n미세 토르소 조절이 필요할 시에 완벽한 피팅감을 느낄 수 있는\n편안함을 제공합니다. (개인적으로 가장 좋았던 부분)\n\n✔️어깨부분의 폼은 40리터급에서는 특별히 신경을 쓴\n편안함을 느낄 수 있었습니다!\n\n개인적으로 툴레배낭은 돌바닥에 깔고 의자로 사용해도, 패킹된 상태로 어느 부분을 잡더라도, 그 내구성에 대해서는 이미 수년간 저희집에서는 검증이 된 배낭이죠🤗\n\n당신의 소중한 무언가를 지켜줄 또 하나

# DB저장 

In [23]:
conn_str = "host=localhost dbname=Instagram user=postgres password=admin port = 5432" # dbname을 만들면 들어간다. port와 password는 변경가능
try :
    conn = psycopg2.connect(conn_str)
    print("=== 접속 성공 =====")
except psycopg2. DatabaseError as db_err:
    print("접속오류 !!")
    print(db_err)

=== 접속 성공 =====


In [24]:
cur = conn.cursor() #conn = 위에 나오듯 포스트그리 커넥션으로 연결을 말한다.
try:
    cur.execute('create table if not exists INSTA (num serial, top_post varchar(3000));')
    conn.commit()                         #테이블 생성
    print('생성 성공')
except :
    conn.rollback()
    print('---DB error')

생성 성공


In [25]:
for i in range(len(result)):
    try :
        cur.execute('INSERT INTO INSTA (num,top_post) VALUES (%s, %s);',
        ((i+1),result[i]))                         # 4개의 column에 번호와 연재일,제목,url추가
        conn.commit()
        print("----- insert ok--")                                     
    except :
        conn. rollback()
        print("--- DB error")

        
# post = []        #모두 진행후 리스트 내용 제거

----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--


# 아래는 아직 무쓸모인 코드들

In [6]:



reallink = []

while True: # 반복문 시작
    pageString = driver.page_source
    bs = BeautifulSoup(pageString, "lxml")

# 게시물 정보 
    for link1 in bs.find_all(name="div",attrs={"class":"Nnq7C weEfm"}):
        title = link1.select('a')[0] 
        real = title.attrs['href']
        reallink.append(real) 
        title = link1.select('a')[1] 
        real = title.attrs['href']
        reallink.append(real) 

# 페이지 스크롤
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
            
        else:
            last_height = new_height
            continue

In [124]:
post_index = []   # 포스트를 얻어온 것에 인덱스를 붙여 파악하려고 했으나 데이터 프레임을 만드니 필요가 없다...
for index, value in enumerate(post):
    post_index.append(index+1)
post_index

a=list(zip(post_index,post))
a

[(1,
  '요즘 트러블 없이 요로코롬\n평화롭게 지내던 춘삼이와 나🤷🏻\u200d♀️🤷🏻\u200d♂️\n\n너무 평화롭다 싶더니\n아니나 다를까 그제 냉삼 먹다가\n한바탕 제대로 했다\n\n씩씩 거리며 카페로 가출했는데\n코젤이 사진 보내며 살살 꼬시는 천춘삼님,,🥸\n어유 증말,,,,,\n\n담엔 아이패드랑 야침들구\n센팍으로 가출해야겠다😎'),
 (2,
  '네이처하이크 TPU 에어매트\n📣DM 톡톡문의\n\n✔️프로필상단링크클릭\n\n#하드캐리몰 #하드캐리 #구매대행 #해외배송 #스마트스토어 #빅파워스토어 #중국배송 #샤오미 #네이처하이크 #캠핑 #차박 #타프 #텐트 #에어텐트#에어매트 #발포매트 #텐트매트 #차박매트 #두꺼운캠핑매트 #네이처하이크에어매트 #백패킹매트 #캠핑매트용품 #에어매트리스'),
 (3,
  '最近日中の写真練習中😆\n\nんで皿に乗せたホットサンドを @ninopeaks さんのヘキサテーブルに載せたらイマイチninoテーブルの良さが隠れる感じやったから直接テーブルにホットサンドのせたらばっちり🥰\nうん、ばっちり妻に怒られたよ😖\n\n\nアンバサダー\n@camp.outdoor.jp\n@gorim5050 @ninopeaks\n@standardthestorage #CAMPONアンバサダー #campgear #キャンプ #camping #アウトドア #outdoor #自然 #田舎 #フュアハンド #DIETZ #ペトロマックス #camp #焚き火 #followme #캠핑 #キャンプ好きな人とつながりたい #ファミキャン #冬キャンプ #ランタン #雰囲気 #サバティカル #ビンテージランタン #スカイパイロットtc #ガレージブランド #CIRCUS #サーカス720\n#ホットサンド #ヨーデルめんたいスパイス\n@yodell.outdoor たっぷりのホットサンド'),
 (4, '출근해라냥~ 오늘도화이팅\n#니팔자가부럽소 #화이팅 #일상 #야옹이 #고양이 #집사 #캠핑 #캠핑용품 #캠퍼캠핑'),
 (5,
  '#꽃등심 #싱싱정육점 #한우\n✔️ 한우 1++ 꽃등심 (기름손질깔끔하게)

### 데이터프레임화하여 엔터를 replace. 추후 변경가능

In [27]:
df = pd.DataFrame(result, columns = ['post']) # 현재 본문글만 긁어서 저장만 했음
df

,post
0,*広角で撮るとものすごい広いテントに見えますね😳GWは昨年に続き雨予報…キャンプ行ってもまた...
1,"#협찬”툴레의 관심은 늘한가지,사람들이 아끼는 것을 옮길 때 그것을 도와주는 무언가..."
2,お昼はバーベキュー☺️那珂湊おさかな市場で美味しいホタテも買って😋美味しかったー❤️❤️#c...
3,내가 좋아하는 하루아마 내가 100번이상 본 것 같다.와이프는 일기쓰는 마음이었다 ...
4,내가 좋아하는 하루아마 내가 100번이상 본 것 같다.와이프는 일기쓰는 마음이었다 ...
5,🏕 #지쁘_캠핑장소개저희가 이번주에 다녀온 캠핑장은경기도 양평의 숲속 계곡 캠핑장{...


In [28]:
df['post']=df['post'].str.replace('\n', ' ')
df

,post
0,*広角で撮るとものすごい広いテントに見えますね😳GWは昨年に続き雨予報…キャンプ行ってもまた...
1,"#협찬”툴레의 관심은 늘한가지,사람들이 아끼는 것을 옮길 때 그것을 도와주는 무언가..."
2,お昼はバーベキュー☺️那珂湊おさかな市場で美味しいホタテも買って😋美味しかったー❤️❤️#c...
3,내가 좋아하는 하루아마 내가 100번이상 본 것 같다.와이프는 일기쓰는 마음이었다 ...
4,내가 좋아하는 하루아마 내가 100번이상 본 것 같다.와이프는 일기쓰는 마음이었다 ...
5,🏕 #지쁘_캠핑장소개저희가 이번주에 다녀온 캠핑장은경기도 양평의 숲속 계곡 캠핑장{...


##### 댓글 관련

In [ ]:
# 게시글의 맨 처음 댓글
driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[1]/div/li/div/div/div[2]/div[1]/span').text

'일단 레쓰고고이죠 🤭🤭🥰🥰'

In [ ]:
# 맨 처음 댓글의 답글남기기 텍스트
driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[1]/li/ul/div/li/div/div/div[2]/div[1]/span').text

'@yunsu880925 맨투맨 입기도 전에 반팔을 꺼내봅니다....🥲'